In [6]:
import os
from dotenv import load_dotenv
from pipelines import get_azure_ml_client

load_dotenv()

datastores_uri = os.getenv("DATASTORES_URI")

ml_client = get_azure_ml_client()

# cluster_name = "sams-16core-DS5-v2"
cluster_name = "sams-f64"
# cluster_name = "sams-f32"
# cluster_name = "sam-standard-d23-v2"
print(ml_client.compute.get(cluster_name))


# from azure.ai.ml.entities import ComputeInstance

# ml_client.compute.begin_start(name="sams-f64").result()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


created_on: 2025-05-11T18:58:41.463804+0000
enable_node_public_ip: true
enable_os_patching: false
enable_root_access: true
enable_sso: true
id: /subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourceGroups/continuous_review_ms_and_ucl/providers/Microsoft.MachineLearningServices/workspaces/EPPI_DEV/computes/sams-f64
idle_time_before_shutdown: PT60M
idle_time_before_shutdown_minutes: 60
last_operation:
  operation_name: Start
  operation_status: Succeeded
  operation_time: '2025-05-12T13:42:19.261Z'
  operation_trigger: User
location: westeurope
name: sams-f64
network_settings:
  private_ip_address: 10.0.0.4
  public_ip_address: 20.16.72.103
os_image_metadata:
  current_image_version: 25.04.22
  is_latest_os_image_version: false
  latest_image_version: 25.04.07
provisioning_state: Succeeded
release_quota_on_stop: false
services:
- display_name: Jupyter
  endpoint_uri: https://sams-f64.westeurope.instances.azureml.ms/tree/
- display_name: Jupyter Lab
  endpoint_uri: https://sams-f64.

In [7]:
from azure.ai.ml import Input, load_component
from azure.ai.ml.dsl import pipeline

# find_single_model = ml_client.components.get(
#     "find_single_model_for_classifier_workbench"
# )

find_single_model = load_component(
    source="./final_components/find_single_model/find_single_model.yml"
)

input_data = ml_client.data.get(name="debunking_review_data", version="1.0.0")

hparam_search_ranges = Input(
    path="./user_inputs/uri_folders/hyperparameter_search_ranges/", type="uri_folder"
)


@pipeline(default_compute=cluster_name, name="find_single_model_pipeline")
def find_single_model_pipeline(
    labelled_data,
    unlabelled_data,
    hparam_search_ranges,
    title_header,
    abstract_header,
    label_header,
    positive_class_value,
    model_name,
    max_n_search_iterations,
    nfolds,
    num_cv_repeats,
    timeout,
    use_early_terminator,
    max_stagnation_iterations,
    wilcoxon_trial_pruner_threshold,
    use_worse_than_first_two_pruner,
    shap_num_display,
):
    find_run = find_single_model(
        labelled_data=labelled_data,
        unlabelled_data=unlabelled_data,
        title_header=title_header,
        abstract_header=abstract_header,
        label_header=label_header,
        positive_class_value=positive_class_value,
        model_name=model_name,
        hparam_search_ranges=hparam_search_ranges,
        max_n_search_iterations=max_n_search_iterations,
        nfolds=nfolds,
        num_cv_repeats=num_cv_repeats,
        timeout=timeout,
        use_early_terminator=use_early_terminator,
        max_stagnation_iterations=max_stagnation_iterations,
        wilcoxon_trial_pruner_threshold=wilcoxon_trial_pruner_threshold,
        use_worse_than_first_two_pruner=use_worse_than_first_two_pruner,
        shap_num_display=shap_num_display,
    )

    return find_run.outputs


first_pipeline = find_single_model_pipeline(
    labelled_data=input_data,
    unlabelled_data=input_data,
    hparam_search_ranges=hparam_search_ranges,
    title_header="title",
    abstract_header="abstract",
    label_header="included",
    positive_class_value=1,
    model_name="lightgbm",
    max_n_search_iterations=20,
    nfolds=3,
    num_cv_repeats=1,
    timeout=None,
    use_early_terminator=False,
    max_stagnation_iterations=None,
    wilcoxon_trial_pruner_threshold=None,
    use_worse_than_first_two_pruner=False,
    shap_num_display=20,
)

import datetime

t = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(t)

first_pipeline.outputs.unlabelled_tfidf_scores.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/unlabelled_tfidf_scores"
first_pipeline.outputs.labelled_tfidf_scores.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/labelled_tfidf_scores"
first_pipeline.outputs.labels.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/labels"
first_pipeline.outputs.best_hparams.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/best_hparams"
first_pipeline.outputs.trained_model.path = f"{datastores_uri}/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/{t}/trained_model"


first_pipeline_job = ml_client.jobs.create_or_update(
    first_pipeline, experiment_name="find_model_sdk"
)


print(f"current time: {datetime.datetime.now()}")

2025-05-12-14-46-09


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute o

current time: 2025-05-12 14:46:16.083087


In [8]:
# Wait until the job completes
ml_client.jobs.stream(first_pipeline_job.name)

RunId: upbeat_tail_h0cdc72y5g
Web View: https://ml.azure.com/runs/upbeat_tail_h0cdc72y5g?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev

Streaming logs/azureml/executionlogs.txt

[2025-05-12 13:46:18Z] Submitting 1 runs, first five are: e161d26f:b1dd119a-9a98-4abe-abaa-44f08936be78
[2025-05-12 13:50:40Z] Completing processing run id b1dd119a-9a98-4abe-abaa-44f08936be78.

Execution Summary
RunId: upbeat_tail_h0cdc72y5g
Web View: https://ml.azure.com/runs/upbeat_tail_h0cdc72y5g?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev



In [4]:
# Get the completed job details
completed_job = ml_client.jobs.get(first_pipeline_job.name)

output_path = completed_job.outputs["labels"].path
print("Output path:", output_path)

Output path: azureml://subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/2025-05-06-21-13-22/labels


In [9]:
output = ml_client.jobs.download(
    first_pipeline_job.name, download_path="pipeline_outputs/", all=True
)

In [12]:
# FINISH THE SECOND PIPELINE WHEN BACK
threshold = -5

In [13]:
from azure.ai.ml import Input

threshold_model_analysis = load_component(
    source="./final_components/threshold_model_analysis/threshold_model_analysis.yml"
)

labelled_tfidf_scores = Input(
    path=first_pipeline.outputs.labelled_tfidf_scores.path, type="uri_folder"
)
labels = Input(path=first_pipeline.outputs.labels.path, type="uri_folder")
model_params = Input(path=first_pipeline.outputs.best_hparams.path, type="uri_folder")


@pipeline(default_compute=cluster_name, name="threshold_model_analysis_pipeline")
def threshold_model_analysis_pipeline(
    labelled_tfidf_scores,
    labels,
    model_name,
    model_params,
    threshold,
    nfolds,
    histogram_num_cv_repeats,
    confusion_num_cv_repeats,
):
    analysis_run = threshold_model_analysis(
        labelled_tfidf_scores=labelled_tfidf_scores,
        labels=labels,
        model_name=model_name,
        model_params=model_params,
        threshold=threshold,
        nfolds=nfolds,
        histogram_num_cv_repeats=histogram_num_cv_repeats,
        confusion_num_cv_repeats=confusion_num_cv_repeats,
    )

    return analysis_run.outputs


second_pipeline = threshold_model_analysis_pipeline(
    labelled_tfidf_scores=labelled_tfidf_scores,
    labels=labels,
    model_name="lightgbm",
    model_params=model_params,
    threshold=threshold,
    nfolds=3,
    histogram_num_cv_repeats=100,
    confusion_num_cv_repeats=1,
)

second_pipeline_job = ml_client.jobs.create_or_update(
    second_pipeline, experiment_name="find_model_sdk"
)

import datetime

print(f"current time: {datetime.datetime.now()}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


current time: 2025-05-12 14:54:11.938778


In [14]:
ml_client.jobs.stream(second_pipeline_job.name)

RunId: shy_balloon_x7ljs2c1t2
Web View: https://ml.azure.com/runs/shy_balloon_x7ljs2c1t2?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev

Streaming logs/azureml/executionlogs.txt

[2025-05-12 13:54:15Z] Submitting 1 runs, first five are: 349d3567:4536ca0a-4c06-4dbb-af8a-ee910baa0ae2


In [ ]:
output = ml_client.jobs.download(
    second_pipeline_job.name,
    download_path="pipeline_outputs/threshold_model_analysis",
    all=True,
)

In [22]:
from azure.ai.ml import Input

classify_unlabelled_tfidf = load_component(
    source="./final_components/classify_unlabelled_tfidf/classify_unlabelled_tfidf.yml"
)

unlabelled_tfidf_scores = Input(
    path=first_pipeline.outputs.unlabelled_tfidf_scores.path, type="uri_folder"
)
trained_model = Input(path=first_pipeline.outputs.trained_model.path, type="uri_folder")


@pipeline(default_compute=cluster_name, name="classify_unlabelled_tfidf_pipeline")
def classify_unlabelled_tfidf_pipeline(
    unlabelled_tfidf_scores,
    threshold,
    trained_model,
):
    classification_run = classify_unlabelled_tfidf(
        unlabelled_tfidf_scores=unlabelled_tfidf_scores,
        threshold=threshold,
        trained_model=trained_model,
    )

    return classification_run.outputs


third_pipeline = classify_unlabelled_tfidf_pipeline(
    unlabelled_tfidf_scores=unlabelled_tfidf_scores,
    threshold=threshold,
    trained_model=trained_model,
)

third_pipeline_job = ml_client.jobs.create_or_update(
    third_pipeline, experiment_name="classify_unlabelled_tfidf"
)

import datetime

print(f"current time: {datetime.datetime.now()}")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


current time: 2025-05-07 14:01:58.219593


In [23]:
ml_client.jobs.stream(third_pipeline_job.name)

RunId: willing_pin_ly24qd8rcx
Web View: https://ml.azure.com/runs/willing_pin_ly24qd8rcx?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev

Streaming logs/azureml/executionlogs.txt

[2025-05-07 13:02:01Z] Submitting 1 runs, first five are: 28dafbf4:77765675-5547-4e67-ba7a-db5116f24bb3
[2025-05-07 13:08:06Z] Completing processing run id 77765675-5547-4e67-ba7a-db5116f24bb3.

Execution Summary
RunId: willing_pin_ly24qd8rcx
Web View: https://ml.azure.com/runs/willing_pin_ly24qd8rcx?wsid=/subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/eppi_dev



In [21]:
first_pipeline.outputs.trained_model.path

'azureml://subscriptions/56539498-d3d8-4a3b-92f4-f3b098a11d1e/resourcegroups/continuous_review_ms_and_ucl/workspaces/EPPI_DEV/datastores/workspaceblobstore/paths/azureml/outputs/2025-05-06-21-13-22/trained_model'